# import python modules

In [13]:
#python code...

# helper functions (if necessary)

In [14]:
#python code...

# read in training data 

In [15]:
#python code...

# visualize data

In [16]:
#python code...

# experiment with building classification models

In [17]:
#python code...

# visualize experimental results (accuracy, precision, recall)

In [18]:
#python code...

# Question: What features and classifier parameters work "best"?

answer goes here..

# create "best" classification model on training data

In [19]:
#python code...

# read in test data

In [20]:
#python code...

# use model to make predictions of test data 

In [21]:
#python code...

# visualize test data with predicted labels

In [22]:
#python code...

# write test data and predictions to \<YourLastName>.tsv

In [23]:
#python code...